Toby & Sandip

2015-11-19

Predictive text entry

In [1]:
from collections import namedtuple
import requests

In [2]:
API_KEY = "ef3fb12d-cbe4-44a6-8273-263cf896e87b"

Input: a string representing the latest snippet of text the user has typed.

Output: a list of (text, identifier) tuples ranked by the percent match of the current text to the concept typed.

In [3]:
def query_ncbo(text):
    """Query NCBO with a text snippet using the look ahead
    suggest function and return the response as a JSON object.
    """
    url = "http://data.bioontology.org/search"
    
    params = {
        "apikey" : API_KEY,
        "ontologies" : "MESH",
        "suggest": True,
        "also_search_obsolete": False,
        "q" : text
    }
    
    resp = requests.get(url, params = params)
    assert resp.status_code == requests.codes.ok
    return resp.json()
    
def get_suggestions(text):
    resp = query_ncbo(text)
    
    # if a person wants to type a synonym for a concept,
    # we will return the preferred label as the suggested text
    # which will make any later NER processing much easier
   
    Concept = namedtuple("Concept", ["name", "id"])
    
    suggestions = []
    for concept in resp["collection"]:
        pref_name = concept["prefLabel"]
        identifier = "MESH:{}".format(concept["@id"][-7:])
        
        suggestions.append(Concept(pref_name, identifier))
        
    return suggestions

In [4]:
get_suggestions("tum")

[Concept(name='tum protein, Enterobacteria phage 186', id='MESH:C099332'),
 Concept(name='RacGap50C protein, Drosophila', id='MESH:C502951')]